In [1]:
#!pip install pdl

In [2]:
#from pdl import pdl
#pdl.dogscats(overwrite_download=True)

In [3]:
import keras
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Model
import tensorflow as tf
from keras import backend as K


Using TensorFlow backend.


In [17]:
sess = tf.Session()
K.set_session(sess)

In [5]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [6]:
base_dir = 'data/dogscats'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test1')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [19]:
base_model = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid', name="inferenceLayer"))

# x=keras.layers.Flatten()(base_model.output)
# x=Dense(64, activation="relu")(x)
# preds=Dense(2, activation="softmax", name="inferenceLayer")(x)



/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
model.compile(optimizer=Adam(lr=1e-04), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:

model.fit_generator(train_generator, 
                              steps_per_epoch=100, epochs=10,
                              validation_data=validation_generator,
                              validation_steps=100,
                             verbose=2)

Epoch 1/10
 - 43s - loss: 0.3575 - acc: 0.8805 - val_loss: 0.1437 - val_acc: 0.9540
Epoch 2/10
 - 33s - loss: 0.1463 - acc: 0.9410 - val_loss: 0.0851 - val_acc: 0.9720
Epoch 3/10
 - 33s - loss: 0.1344 - acc: 0.9475 - val_loss: 0.0670 - val_acc: 0.9715
Epoch 4/10
 - 34s - loss: 0.1065 - acc: 0.9565 - val_loss: 0.0776 - val_acc: 0.9740
Epoch 5/10
 - 34s - loss: 0.0944 - acc: 0.9615 - val_loss: 0.0989 - val_acc: 0.9715
Epoch 6/10
 - 34s - loss: 0.1185 - acc: 0.9535 - val_loss: 0.1336 - val_acc: 0.9585
Epoch 7/10
 - 34s - loss: 0.1471 - acc: 0.9430 - val_loss: 0.1613 - val_acc: 0.9500
Epoch 8/10
 - 34s - loss: 0.1191 - acc: 0.9515 - val_loss: 0.0835 - val_acc: 0.9775
Epoch 9/10
 - 34s - loss: 0.1228 - acc: 0.9530 - val_loss: 0.1105 - val_acc: 0.9670
Epoch 10/10
 - 34s - loss: 0.1725 - acc: 0.9480 - val_loss: 1.3944 - val_acc: 0.8605


In [10]:
model.save('resnet50_final.h5')



In [20]:
model.load_weights('resnet50_final.h5')

In [12]:
builder = tf.saved_model.builder.SavedModelBuilder("forGo")
builder.add_meta_graph_and_variables(sess, ["tags"])
builder.save()
#sess.close()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: forGo/saved_model.pb


In [13]:
model.inputs

[<tf.Tensor 'resnet50_input:0' shape=(?, 224, 224, 3) dtype=float32>]

In [14]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [29]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

preds = model.predict(x)

print(preds)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

[[1.]]
